#  Learning Dash

## por Cristian Quintana ## cquintan@cern.ch



Tenemos dos alternativas para la gestión de los paquetes que vamos a necesitar. Ambas estan relacionadas con el papel
que vamos a desempeñar: ¿Seremos los encargados de desarrollar la app? ¿Seremos meros usuarios o testers?

Si nuestro papel es desarrollar la app tendremos que ser nosotros los encargados de instalar todos los paquetes que
consideremos oportunos. Para ello es recomendable crear un nuevo entorno virtual. De esta forma aislaremos el entorno
de trabajo de la app del resto del equipo (más o menos). 
## Generating a new environment and installing packages:

    Let's open a terminal (LINUX)
	Make a dir and cd:
 
mkdir Dash_Jupyter

cd Dash_Jupyter


	Create a new environment and activate (OPTIONAL):
python3 -m venv DashJ

source DashJ/bin/activate

	Install packages:
python -m pip install dash==2.8.1 pandas==1.5.3 jupyter jupyterlab jupyter_dash==0.4.2 dash_bootstrap_components openpyxl datetime 

    Open jupyter lab:
jupyter lab


La otra posibilidad es que la app exista y queramos lanzarla. Si el desarrollador hizo lo propio, ya dispondremos de este entorno
virtual, y nuestra labor será cargarlo.
## Download the app from a repository:
    Primero hay que tener instalado git:
    
sudo apt install git

    Copiar (clonar) el repositorio
    
cd ~ #cd a la carpeta home, o a otra dónde se quiera descargar la app
git clone r



## The script

In [1]:
# IMPORT Packages
from dash import Dash, dcc, html, Input, Output, State # Dash
import dash_bootstrap_components as dbc
from jupyter_dash import JupyterDash   # Dash for Jupyter
import subprocess                      # Download files
import os
import pandas as pd                    # data
import plotly.express as px            # plot easy
import plotly.graph_objects as go      # plot hard
from datetime import date
import gradio as gr                    # gradio dashboard
import numpy as np
import plotly.io as pio
pio.renderers.default = 'notebook'

In [2]:
def CountPixels(input_img):
   # import cv2
   # from PIL import Image
   # img = Image.open(input_img);
    dark_count = np.sum(np.array(input_img) <= 10)
    return  f"El número de pixeles oscuros es {dark_count}"

demo = gr.Interface(CountPixels, gr.Image(), "text")
demo.launch(share = False)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


# EXERCISE: READING FROM Google spreadsheet (excel)

ENLACE: https://docs.google.com/spreadsheets/d/1p_yg1YlVpL7izGE6PcFni9sC4WNzQaMPtMWlO234lmI/edit?usp=sharing

## Download and format

In [3]:
# ------> Excel
# Make the document public 
# copy link 
# Generate download link in: https://www.graytechnical.com/blog/google-drive-online-download-link-generator-files-sheets-and-docs-online/

def download_as(url, name="", dir="Data_files/"):
    # Rename 
    if name != "":
        command = f'wget --output-document={dir+name} {url}'
    else:
        command = f'wget {url} -P {dir}'
        
    subprocess.run(command, shell=True, stderr=subprocess.DEVNULL)
    print("Done! -> Downloaded file")

# DOWNLOAD
filename = "Dashboard_excel.xlsx"
url = "https://docs.google.com/spreadsheets/d/1p_yg1YlVpL7izGE6PcFni9sC4WNzQaMPtMWlO234lmI/export?format=xlsx"
download_as(url, filename)

# FORMAT to dataframe
excel_data = pd.read_excel("Data_files/"+filename, skiprows = 5)
excel_data.head()

Done! -> Downloaded file


,Nombre,Valoracion Curso,Valoracion Dash,Conjunto
0,Alba,1,1,1
1,Helena,3,1,4
2,Alicia,3,3,5
3,Juan,3,3,1
4,Carlos,1,1,4


## Contruyendo la app

In [12]:
header = """ \t Encuesta sobre el curso
         """
style_header = {'margin-left':'500px', 'margin-top':'40px', 'margin-right':'15px', 'margin-bottom':'40px'}

Histo1 = px.histogram(excel_data, x="Valoracion Curso", nbins=5, range_x=[0.5, 5.5])
Histo2 = px.histogram(excel_data, x="Valoracion Dash", nbins=5, range_x=[0.5, 5.5])
Histo3 = px.histogram(excel_data, x="Conjunto",  nbins=5, range_x=[0.5, 5.5])
# go.FigureWidget(Histo1)

def replot(Histo1, Histo2, Histo3, data1, data2, data3):
    Histo1.update_traces(x=data1)
    Histo2.update_traces(x=data2)
    Histo3.update_traces(x=data3)


app = JupyterDash(__name__, external_stylesheets=[dbc.themes.CYBORG])

app.layout = dbc.Container(
    [
        dbc.Row(html.H5(header), style = style_header),
        dbc.Row(
            [
                dbc.Col(dcc.Graph(figure = Histo1, id="H1")),
                dbc.Col(dcc.Graph(figure = Histo2, id="H2")),
                dbc.Col(dcc.Graph(figure = Histo3, id="H3")),
            ]
        ),
         dbc.Row([dbc.Col(html.Button('Actualizar datos', id='update-data', n_clicks=0), width = 2),
                  dbc.Col(html.Div(children="Text", id='update-date'), width = 8),
                 ]),
    ]
)

@app.callback(
    Output('update-date', 'children'),
    Output('H1', 'figure'),
    Output('H2', 'figure'),
    Output('H3', 'figure'),
    Input('update-data', 'n_clicks'),
)
def update_output(n_clicks):
    if n_clicks>0: 
        filename = "Dashboard_excel.xlsx"
        url = "https://docs.google.com/spreadsheets/d/1p_yg1YlVpL7izGE6PcFni9sC4WNzQaMPtMWlO234lmI/export?format=xlsx"
        download_as(url, filename)
        excel_data = pd.read_excel("Data_files/"+filename, skiprows = 5)
        data1, data2, data3 = excel_data["Valoracion Curso"], excel_data["Valoracion Dash"], excel_data["Conjunto"]
        replot(Histo1, Histo2, Histo3, data1, data2, data3)
        text = ("Actualizado! ->  "+subprocess.run("date", shell=True, capture_output=True, text=True).stdout.strip())
    else:
        text = ("Los ficheros de datos no han sido actualizados desde el inicio de la sesión")

    return text, Histo1, Histo2, Histo3


if __name__ == "__main__":
    app.run_server(debug=True)


Dash app running on http://127.0.0.1:8050/


Exception in thread Exception in thread Thread-28 (<lambda>):
Traceback (most recent call last):
  File "/home/cristian/anaconda3/envs/PRG0/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
Exception in thread Thread-10 (<lambda>):
Traceback (most recent call last):
  File "/home/cristian/anaconda3/envs/PRG0/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
Thread-36 (<lambda>):
Traceback (most recent call last):
  File "/home/cristian/anaconda3/envs/PRG0/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/cristian/anaconda3/envs/PRG0/lib/python3.10/threading.py", line 953, in run
    self.run()
  File "/home/cristian/anaconda3/envs/PRG0/lib/python3.10/threading.py", line 953, in run
    self.run()
  File "/home/cristian/anaconda3/envs/PRG0/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/home/cristian/Lectures/Dash_Jupyter/DashJ/lib/python3.10/site-packages/dash/dash.py", l

# EXERCISE: READING FROM WEB (Covid Data)

In [5]:
# DOWNLOAD datafiles
url = 'https://opendata.ecdc.europa.eu/covid19/nationalcasedeath_eueea_daily_ei/csv/data.csv'
command = f'wget {url} -P Data_files/'
subprocess.run(command, shell=True)
subprocess.run('mv Data_files/data.csv Data_files/covid_Europe.csv', shell=True)

--2023-06-22 16:42:55--  https://opendata.ecdc.europa.eu/covid19/nationalcasedeath_eueea_daily_ei/csv/data.csv
Resolving opendata.ecdc.europa.eu (opendata.ecdc.europa.eu)... 88.131.255.63
Connecting to opendata.ecdc.europa.eu (opendata.ecdc.europa.eu)|88.131.255.63|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1697172 (1.6M) [application/octet-stream]
Saving to: ‘Data_files/data.csv’

     0K .......... .......... .......... .......... ..........  3%  361K 4s
    50K .......... .......... .......... .......... ..........  6%  361K 4s
   100K .......... .......... .......... .......... ..........  9%  705K 3s
   150K .......... .......... .......... .......... .......... 12% 24.1M 3s
   200K .......... .......... .......... .......... .......... 15%  743K 2s
   250K .......... .......... .......... .......... .......... 18% 13.9M 2s
   300K .......... .......... .......... .......... .......... 21%  728K 2s
   350K .......... .......... .......... .......... 

CompletedProcess(args='mv Data_files/data.csv Data_files/covid_Europe.csv', returncode=0)

In [6]:
# LOAD Data file(s)

data = (pd.read_csv("Data_files/covid_Europe.csv"))

In [7]:
print(data.head())

      dateRep  day  month  year    cases  deaths countriesAndTerritories  \
0  23/10/2022   23     10  2022   3557.0     0.0                 Austria   
1  22/10/2022   22     10  2022   5494.0     4.0                 Austria   
2  21/10/2022   21     10  2022   7776.0     4.0                 Austria   
3  20/10/2022   20     10  2022   8221.0     6.0                 Austria   
4  19/10/2022   19     10  2022  10007.0     8.0                 Austria   

  geoId countryterritoryCode  popData2020 continentExp  
0    AT                  AUT      8901064       Europe  
1    AT                  AUT      8901064       Europe  
2    AT                  AUT      8901064       Europe  
3    AT                  AUT      8901064       Europe  
4    AT                  AUT      8901064       Europe  


In [8]:
# DATA Selection
data["Date"] = pd.to_datetime(data["dateRep"], format="%d/%m/%Y")

spain_data = data[data["countriesAndTerritories"] == "Spain"]
print(spain_data.head())

          dateRep  day  month  year    cases  deaths countriesAndTerritories  \
26816  21/10/2022   21     10  2022  25422.0    75.0                   Spain   
26817  14/10/2022   14     10  2022  20652.0    68.0                   Spain   
26818  07/10/2022    7     10  2022  10843.0    68.0                   Spain   
26819  04/10/2022    4     10  2022   8114.0    83.0                   Spain   
26820  30/09/2022   30      9  2022  10721.0    95.0                   Spain   

      geoId countryterritoryCode  popData2020 continentExp       Date  
26816    ES                  ESP     47332614       Europe 2022-10-21  
26817    ES                  ESP     47332614       Europe 2022-10-14  
26818    ES                  ESP     47332614       Europe 2022-10-07  
26819    ES                  ESP     47332614       Europe 2022-10-04  
26820    ES                  ESP     47332614       Europe 2022-09-30  


In [9]:
regions = data["countriesAndTerritories"].sort_values().unique()
print(regions)

['Austria' 'Belgium' 'Bulgaria' 'Croatia' 'Cyprus' 'Czechia' 'Denmark'
 'Estonia' 'Finland' 'France' 'Germany' 'Greece' 'Hungary' 'Iceland'
 'Ireland' 'Italy' 'Latvia' 'Liechtenstein' 'Lithuania' 'Luxembourg'
 'Malta' 'Netherlands' 'Norway' 'Poland' 'Portugal' 'Romania' 'Slovakia'
 'Slovenia' 'Spain' 'Sweden']


In [10]:
#--------------- Parameters ----------------------------------------------------------------------------
# FIGURES
WH_set1 = [900,400]

figs = {}
#
figs["CasesAndDeaths"] = go.Figure() # Plot cases and Deaths vs date
figs["CasesCountries"] = go.Figure()

#--------------- Functions -----------------------------------------------------------------------------

""" Crear una función que me permita descargar el set de datos de una url en un directorio determinado
    Si se le aporta como argumento un nombre, el fichero descargado es renombrado
    Opcional: Return o print mensaje "Compleatado"
""" 
def download_as(url, name="", dir="Data_files/"):
    # Rename 
    if name != "":
        command = f'wget --output-document={dir+name} {url}'
    else:
        command = f'wget {url} -P {dir}'
        
    subprocess.run(command, shell=True, stderr=subprocess.DEVNULL)
    print("Done! -> Downloaded file")
    

""" La función lee el fichero de datos con pandas.
    Como primer argumento un str con el nombre de un país -> sub dataset del país
    Como segundo argumento lista de str de paises -> lista de sub datasets
    retorna datos completos, datos del país, datos de los paises  (lista)
"""
def get_datasets(Country = "Spain", Countries = ["Spain","France","Germany"]):
    # Leer csv
    data = (pd.read_csv("Data_files/covid_Europe.csv"))
    # Crear columna "Date" con formato datetime
    data["Date"] = pd.to_datetime(data["dateRep"], format="%d/%m/%Y")
    # Generar dataset y lista de datasets
    country_data = data[data["countriesAndTerritories"] == Country]
    countries_data = [data[data["countriesAndTerritories"] == _c] for _c in Countries]

    return data, country_data, countries_data

""" Función que genera las figuras con PLOTLY!
    Tiene como argumentos los mismos argumentos que la anterior además de el return de la anterior
    country (str), country_data (dataset), countries [str], countries_data [datasets]
"""
def gen_figs(country, country_data, countries, countries_data):
    figs = {}
    
    # Figures of one single country (First section)
    # Figure CasesAndDeaths
    figs["CasesAndDeaths"] = go.Figure()
    figs["CasesAndDeaths"].add_trace(go.Scatter(x=country_data["Date"], y=country_data["cases"]/country_data["cases"].max(),
                        mode='lines',
                        name='Cases'))
    figs["CasesAndDeaths"].add_trace(go.Scatter(x=country_data["Date"], y=country_data["deaths"]/country_data["deaths"].max(),
                        mode='lines',
                        name='Deaths'))
    figs["CasesAndDeaths"].update_layout(
                       xaxis_title='Date',
                       yaxis_title='Normallized to max',
                       title='Visualizando '+country)

    # Figures of one multiple countries (Second section)
    # Figure CasesCountries
    figs["CasesCountries"] = go.Figure()
    N_countries = len(countries)
    if N_countries>0:
        for i in range(N_countries):
            tdata = countries_data[i]
            figs["CasesCountries"].add_trace(go.Scatter(x=tdata["Date"], y=tdata["cases"],
                        mode='lines',
                        name=countries[i]))

    return figs

#--------------- Building the app ------------------------------------------------------------------------------

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.MINTY])

header = """Dashboard interactivo con los últimos datos de Covid para Europa hasta final del seguimiento. 
         En los diferentes apartados podrás acceder a los datos individuales de cada país europeo así como realizar comparaciones entre ellos
         o seleccionar el rango de fechas a estudiar.
         """

style_1 = {'margin-left':'15px', 'margin-top':'10px', 'margin-right':'15px', 'margin-bottom':'15px'}

app.layout = html.Div(
    [
        dbc.Row(dbc.Col(html.Img(src="assets/LogoDash.png"), width = 10)),
        dbc.Row(dbc.Col(html.H5(header), width = 10), style = style_1),
        dbc.Row([dbc.Col(html.Button('Actualizar datos', id='update-data', n_clicks=0),
                         style = style_1, width = 1),
                 dbc.Col(html.Div(children="", id='update-date'),
                         style = style_1, width = 9),
                 
                ]),
        dbc.Row(dbc.Col(html.Img(src="assets/LineDash.png"), width = 10)),

        dbc.Row(
            [
                dbc.Col([dcc.Dropdown(
                            id="region-filter",
                            options=[
                                {"label": region, "value": region}
                                for region in regions
                            ],
                            value="Spain",
                            clearable=False,
                            className="dropdown",
                        ), 
                        dcc.DatePickerRange(
                        id='datePicker',
                        min_date_allowed=date(2020, 1, 1),
                        max_date_allowed=date(2022, 10, 19),
                        start_date=date(2020, 1, 1),
                        end_date=date(2022, 10, 19),
                        ),
                        ],
                        width = 3,style = style_1),
                dbc.Col(dcc.Graph(figure = figs["CasesAndDeaths"], id="F_CD"), width = 7)
            ]
        ),
        dbc.Row(dbc.Col(html.Img(src="assets/LineDash.png"), width = 10)),

        dbc.Row(
            [
                dbc.Col(dcc.Dropdown(
                            id="countries-filter",
                            options=[
                                {"label": region, "value": region}
                                for region in regions
                            ],
                            value=["Spain", "Portugal", "Germany"],
                            multi=True,
                            clearable=False,
                            className="dropdown",
                        ), 
                        width = 3,style = style_1),
                dbc.Col(dcc.Graph(figure = figs["CasesCountries"], id="F_CC"), width = 7)
            ]
        ),

        dbc.Row(dbc.Col(html.Img(src="assets/LineDash.png"), width = 10)),


    ]
)

@app.callback(
    Output('update-date', 'children'),
    Output('update-data', 'n_clicks'),
    Output('F_CD', 'figure'),
    Output('F_CC', 'figure'),
    Input('update-data', 'n_clicks'),
    Input('region-filter', 'value'),
    Input('countries-filter', 'value'),
    Input('datePicker', 'start_date'),
    Input('datePicker', 'end_date')
)
def update_output(n_clicks, region, countries, start_date, end_date):
    if not isinstance(countries, list):
        countries = [countries]
    
    if n_clicks > 0: 
        url = 'https://opendata.ecdc.europa.eu/covid19/nationalcasedeath_eueea_daily_ei/csv/data.csv'
        download_as(url, name = "covid_Europe.csv")
        text = "Actualizado! -> "+subprocess.run("date", shell=True, capture_output=True, text=True).stdout.strip()
    else:
        text = "Los ficheros de datos no han sido actualizados desde el inicio de la sesión"


    data, country_data, countries_data = get_datasets(Country = region, Countries = countries)

    if start_date != None and end_date != None:
        country_data = country_data[(country_data['Date'] >= start_date) & (country_data['Date'] <= end_date)]
    
    figs = gen_figs(region, country_data, countries, countries_data)
    #return text, 0, figs["CasesAndDeaths"], figs["CasesCountries"]
    return text, 0, *figs.values()


#--------------- Launch/Update the app ------------------------------------------------------------------------------
if __name__ == "__main__":
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


# Biblio

https://dash-bootstrap-components.opensource.faculty.ai/docs/components/layout/